# Imports

In [1]:
import re
import string
import unicodedata
import pickle
from collections import Counter
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from scipy.sparse import hstack

import contractions

import nltk
from nltk.corpus import stopwords

import spacy

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

from sklearn import (
    linear_model,
    feature_extraction,
    model_selection,
    naive_bayes,
    metrics,
    ensemble,
)

import pyLDAvis
import pyLDAvis.gensim_models

from pprint import pprint

## Data

In [2]:
data = pd.read_csv("data/final_all_data.csv")
data["full_text"] = data.apply(lambda x: x["title"] + " " + x["selftext"], axis=1)
data.shape


(127579, 11)

In [3]:
data

,author,selftext,created_utc,subreddit,subreddit_subscribers,title,date,year-month,post_length,depression,full_text
0,--PunPun--,I'm a relatively new Evelynn player and right ...,1587305577,EvelynnMains,14681,Has anyone made a detailed spreadsheet of her ...,2020-04-19 10:12:57,2020-04,51,1.0,Has anyone made a detailed spreadsheet of her ...
1,--jyushimatsudesu,"""my arm hurt this morning and i prayed and now...",1552057727,exchristian,48153,so self centered,2019-03-08 10:08:47,2019-03,88,1.0,"so self centered ""my arm hurt this morning and..."
2,--jyushimatsudesu,"(warning : very sappy, i'm sorry, she's just g...",1552258247,teenagers,1226850,letter to my crush ♥,2019-03-10 18:50:47,2019-03,186,1.0,"letter to my crush ♥ (warning : very sappy, i'..."
3,--jyushimatsudesu,"being watched, 24/7?? like, imagine if someone...",1553425235,exchristian,48201,do Christians actually think about this?,2019-03-24 07:00:35,2019-03,117,1.0,do Christians actually think about this? being...
4,--jyushimatsudesu,i don't mean what happens when you feel you're...,1554040237,TooAfraidToAsk,578671,who else suddenly jolts awake from a light sle...,2019-03-31 09:50:37,2019-03,51,1.0,who else suddenly jolts awake from a light sle...
...,...,...,...,...,...,...,...,...,...,...,...
127574,zzspecialty,"I'm a 5' 11 23 year old, average weight and se...",1599439665,dirtyr4r,542148,23 [M4F]- Florida- Casual flirting/open relati...,2020-09-06 20:47:45,2020-09,89,0.0,23 [M4F]- Florida- Casual flirting/open relati...
127575,zzspecialty,So let's skip the shaming of cheating. We all ...,1600748511,sex,1882945,The urge to have sexual relations with another...,2020-09-22 00:21:51,2020-09,168,0.0,The urge to have sexual relations with another...
127576,zzzgfr,Early this morning I woke up and hallucinated/...,1564619771,Sleepparalysis,16800,Is it possible to be only partially paralyzed?,2019-07-31 20:36:11,2019-08,79,0.0,Is it possible to be only partially paralyzed?...
127577,zzzgfr,I attempted to force myself into sp the other ...,1565838037,Sleepparalysis,16688,Should my body tingle before entering paralysis,2019-08-14 23:00:37,2019-08,79,0.0,Should my body tingle before entering paralysi...


In [4]:
# Drop strange subreddit that is very represented in the data but has only 4.2k members.
# data = data.loc[data["subreddit"] != "randonaut_reports"].reset_index(drop=True)

# Drop any leakage in the data from depression-related subreddits
data = data.loc[
    ~data["subreddit"].isin(
        [
            "randonaut_reports",
            "depression",
            "SuicideWatch",
            "depression_help",
            "depressed",
        ]
    )
].reset_index(drop=True)

data.shape


(104061, 11)

In [5]:
data.describe()


,created_utc,subreddit_subscribers,post_length,depression
count,1.040610e+05,1.040610e+05,104061.000000,104061.000000
mean,1.576417e+09,7.858885e+05,117.344115,0.461354
std,1.835945e+07,2.545090e+06,84.472321,0.498507
min,1.546301e+09,0.000000e+00,30.000000,0.000000
25%,1.560321e+09,3.248600e+04,54.000000,0.000000
50%,1.576567e+09,1.408550e+05,88.000000,0.000000
75%,1.591326e+09,5.983100e+05,155.000000,1.000000
max,1.609459e+09,2.994312e+07,408.000000,1.000000


## Train-Val-Test Split

In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    data["full_text"], data["depression"], test_size=0.10, random_state=42
)
X_train, X_val, y_train, y_val = model_selection.train_test_split(
    X_train, y_train, test_size=0.10, random_state=42
)

print("Train, Val, Test Sizes")
print(y_train.shape, y_val.shape, y_test.shape)


Train, Val, Test Sizes
(84288,) (9366,) (10407,)


In [7]:
X_train

13578    I’m giving up on my dreams b/c I am too sick &...
54334    INTJs and morality Others and I consider mysel...
28263    Im at crossroads and idk what to do 😣 I have b...
92109    What does it feel like to trip in a forest or ...
3561     Hell. So fuckin’ hot and there is little escap...
                               ...                        
63858    Are there any erasable pens that I don’t have ...
24796    Anyone heard of leaky gut? It all started few ...
70917    Long Distance Relationship Me(16M) and Friend/...
63875    Do you think Life Orientation is a useless sub...
86581    Does anyone else mimic symptoms because of hea...
Name: full_text, Length: 84288, dtype: object

In [8]:
print(X_val.iloc[0])


Are there cash cards of any type at gas stations? When I discovered my bf’s cam girl addiction, I went through his bank accounts. I found cash app transactions from march-April that he admitted were for porn. Then, there were two other transactions on another card from February that he admitted were for porn. That’s it! I’ve gone back and back and can not find another transaction that could be attributed to porn. It makes no sense. 

While reviewing his transactions, I realized he was spending 1k every month at gas stations. He insisted it was food, gas, energy drinks, and cigarettes. Sometimes there are multiple stops in one day, and it’s multiple days in a row. He agreed he had a spending problem and would fill up and buy a carton once a week at Walmart. 

I checked today, and the frequent stops are back. He’s spent $700 in the last month. So I’m wondering if he’s purchasing some kind of card that he pays for porn with.

Spending so much at gas stations doesn’t make sense. And neithe

## Preprocessing

In [9]:
translate_table = dict((ord(char), None) for char in string.punctuation + "‘’")
# Pre-processing code comes from modification of Gensim's simple_tokenize to include numbers and remove punctuation
# https://github.com/RaRe-Technologies/gensim/blob/351456b4f7d597e5a4522e71acedf785b2128ca1/gensim/utils.py#L288


def to_unicode(text):
    if isinstance(text, str):
        return text
    return str(text, "ut8")


def preprocess(text):
    text = to_unicode(text)
    if lowercase:
        text = text.lower()

    if deaccent:
        text = unicodedata.normalize("NFD", text)
        text = "".join(ch for ch in text if unicodedata.category(ch) != "Mn")
        text = unicodedata.normalize("NFC", text)

    # Remove /r/depression
    if remove_leaks:
        text = re.sub(r"\/?r\/?depression", "", text)
    # Remove URL
    if remove_urls:
        text = re.sub(r"https?:\/\/\S+", "", text)

    if remove_reddit_chars:
        text = re.sub(r"\/r\/|\/u\/", "", text)

    if expand_contractions:
        text = contractions.fix(text)

    # Remove punctuation
    if remove_punctuation:
        text = text.replace("-", " ")
        text = text.translate(translate_table)

    text = text.split()

    if remove_numbers:
        text = [w for w in text if not w.isdigit()]

    return [token for token in text if min_len <= len(token) and max_len >= len(token)]


In [10]:
min_len = 2
max_len = 15

lowercase = True
deaccent = True
remove_leaks = True
remove_urls = True
remove_reddit_chars = True
expand_contractions = True
remove_punctuation = True
remove_numbers = False


In [11]:
X_train_clean = list(map(preprocess, X_train))
X_val_clean = list(map(preprocess, X_val))
X_test_clean = list(map(preprocess, X_test))


In [12]:
print(X_val_clean[0])


['are', 'there', 'cash', 'cards', 'of', 'any', 'type', 'at', 'gas', 'stations', 'when', 'discovered', 'my', 'bfs', 'cam', 'girl', 'addiction', 'went', 'through', 'his', 'bank', 'accounts', 'found', 'cash', 'app', 'transactions', 'from', 'march', 'april', 'that', 'he', 'admitted', 'were', 'for', 'porn', 'then', 'there', 'were', 'two', 'other', 'transactions', 'on', 'another', 'card', 'from', 'february', 'that', 'he', 'admitted', 'were', 'for', 'porn', 'that', 'is', 'it', 'have', 'gone', 'back', 'and', 'back', 'and', 'can', 'not', 'find', 'another', 'transaction', 'that', 'could', 'be', 'attributed', 'to', 'porn', 'it', 'makes', 'no', 'sense', 'while', 'reviewing', 'his', 'transactions', 'realized', 'he', 'was', 'spending', '1k', 'every', 'month', 'at', 'gas', 'stations', 'he', 'insisted', 'it', 'was', 'food', 'gas', 'energy', 'drinks', 'and', 'cigarettes', 'sometimes', 'there', 'are', 'multiple', 'stops', 'in', 'one', 'day', 'and', 'it', 'is', 'multiple', 'days', 'in', 'row', 'he', 'agr

### Make n-grams

In [13]:
# Drop any terms that do not appear in at least 0.1% of documents
min_docfreq = int(X_train.shape[0] * 0.0005)
print("Min Doc Frequency: ", min_docfreq)
ngram_threshold = 1
make_bigrams = True
make_moregrams = True
remove_stopwords = True


Min Doc Frequency:  42


In [14]:
self_counter = {}
for doc in X_test_clean:
    for word in doc:
        self_counter[word] = self_counter.get(word, 0) + 1
len(self_counter)


45073

In [15]:
bigram = Phrases(
    X_train_clean,
    min_count=min_docfreq,
    threshold=ngram_threshold,
    connector_words=ENGLISH_CONNECTOR_WORDS,
)
bigram_mod = gensim.models.phrases.Phraser(bigram)


KeyboardInterrupt: 

In [ ]:
len(bigram_mod.phrasegrams)


In [ ]:
moregram = Phrases(
    bigram[X_train_clean],
    min_count=min_docfreq,
    threshold=ngram_threshold,
    connector_words=ENGLISH_CONNECTOR_WORDS,
)
moregram_mod = gensim.models.phrases.Phraser(moregram)


In [ ]:
len(moregram_mod.phrasegrams)


In [ ]:
X_train_ngram = moregram_mod[X_train_clean]
X_val_ngram = moregram_mod[X_val_clean]
X_test_ngram = moregram_mod[X_test_clean]


In [ ]:
print(X_val_ngram[0])


### Remove Stopwords

In [ ]:
my_stopwords = stopwords.words("english")  # Common terms that did not seem helpful.


def remove_stopwords(str_):
    if str_ in my_stopwords:
        return False
    return True


stopless_text = [list(filter(remove_stopwords, t)) for t in X_train_ngram]


In [ ]:
all_text = []
for t in stopless_text:
    all_text.extend(t)
c = Counter(all_text)
print([x[0] for x in c.most_common()[:100]])


In [ ]:
## Manually selected from every common terms that do not seem to provide too much information
extra_stopwords = [
    "do_not",
    "it_is",
    "like",
    "get",
    "one",
    "know",
    "have_been",
    "really",
    "that_is",
    "work",
    "also",
    "even",
    "want",
    "something",
    "this_is",
    "it_was",
    "did_not",
    "if_you",
    "there_is",
    "got",
    "back",
    "go",
    "new",
    "am_not",
    "things",
    "lot",
    "going",
    "does_not",
    "around",
    "would",
    "day",
    "anything",
    "way",
    "someone",
    "but_it",
    "still",
    "without",
    "since",
    "you_are",
    "ampx200b",
    "getting",
    "is_not",
    "need",
    "first",
    "everything",
    "us",
    "but_am",
    "would_be",
    "right",
    "see",
    "take",
    "use",
    "they_are",
    "make",
    "always",
    "what_is",
    "try",
    "looking",
    "maybe",
    "well",
    "start",
    "will_be",
    "so_much",
    "say",
    "actually" "two",
    "how_do",
    "is_it",
    "know_what",
    "another",
]


In [ ]:
my_stopwords = stopwords.words("english") + extra_stopwords


def remove_stopwords(str_):
    if str_ in my_stopwords:
        return False
    return True


X_train_stopless = [list(filter(remove_stopwords, t)) for t in X_train_ngram]
X_val_stopless = [list(filter(remove_stopwords, t)) for t in X_val_ngram]
X_test_stopless = [list(filter(remove_stopwords, t)) for t in X_test_ngram]


In [ ]:
print(X_val_stopless[0])


### Lemmatization + Stem

In [ ]:
def self_lemmatize(str_list):
    spacy_out = nlp(" ".join(str_list))
    lemma_out = [token.lemma_ for token in spacy_out]
    return lemma_out


In [ ]:
X_train_lemma = [self_lemmatize(l) for l in X_train_stopless]


In [ ]:
X_val_lemma = [self_lemmatize(l) for l in X_val_stopless]
X_test_lemma = [self_lemmatize(l) for l in X_test_stopless]


In [ ]:
train_index = X_train.index
val_index = X_val.index
test_index = X_test.index

In [ ]:
train_text = [" ".join(line) for line in X_train_lemma]
val_text = [" ".join(line) for line in X_val_lemma]
test_text = [" ".join(line) for line in X_test_lemma]

In [32]:
new_df = pd.DataFrame()
new_df["real_index"] = np.concatenate((train_index,val_index,test_index))
new_df["cleaned_text"] = train_text + val_text  + test_text

In [33]:
new_df = new_df.sort_values(by="real_index").reset_index(drop=True)
new_df

,real_index,cleaned_text
0,0,has_anyone make detailed spreadsheet matchup l...
1,1,self center my_arm hurt this_morning pray it_d...
2,2,letter my_crush warning sappy am_sorry she_is ...
3,3,christian actually think_about watch 247 imagi...
4,4,who_else suddenly jolt awake light sleep no_re...
...,...,...
104056,104056,23 m4f florida casual flirtingopen relationshi...
104057,104057,urge sexual relation woman date let_us skip sh...
104058,104058,possible partially paralyzed early this_morne ...
104059,104059,my_body tingle enter paralysis attempt force_m...


In [34]:
data["cleaned_text"] = new_df["cleaned_text"]

In [35]:
data.to_csv("data/final_lemmatized_go.csv",index=False)

In [36]:
with open("train-post.pkl", "wb") as file:
    pickle.dump(X_train_lemma, file)

with open("val-post.pkl", "wb") as file:
    pickle.dump(X_val_lemma, file)

with open("test-post.pkl", "wb") as file:
    pickle.dump(X_test_lemma, file)


In [16]:
with open("train-post.pkl", "rb") as file:
    X_train_lemma = pickle.load(file)

with open("val-post.pkl", "rb") as file:
    X_val_lemma = pickle.load(file)

with open("test-post.pkl", "rb") as file:
    X_test_lemma = pickle.load(file)

In [17]:
print(" ".join(X_val_lemma[0]))


are_there cash card any_type gas station discover bfs cam girl addiction went_through bank account find cash app transaction march april that_he admit porn there_were two_other transaction card february that_he admit porn have_gone find_another transaction that_could attribute porn it_makes no_sense review transaction realize he_was spend 1k every_month gas station insist food gas energy drink cigarette sometimes there_are multiple stop one_day multiple day row agree he_had spending problem fill buy carton once_week walmart check today frequent stop he_is spend 700 last_month so_am wondering_if he_is purchase some_kind card that_he pay porn spend gas station make_sense neither no_other transaction account past february when_he is_been doing_this year


In [11]:
data = pd.read_csv("data/redo_clean.csv")
data["cleaned_text2"] = data["cleaned_text2"].apply(lambda x: x.split(" "))

train = data.loc[data["split"] == "train"]
val = data.loc[data["split"] == "val"]
test = data.loc[data["split"] == "test"]

X_train_lemma = train["cleaned_text2"]
y_train = train["depression"]

X_val_lemma = val["cleaned_text2"]
y_val = val["depression"]

X_test_lemma = test["cleaned_text2"]
y_test = test["depression"]


# Modeling

## Bag-Of-Words

### Naive Bayes

In [14]:
def dummy(doc):
    return doc


nb_cv = feature_extraction.text.CountVectorizer(
    min_df=41,
    binary=True,
    tokenizer=dummy,
    preprocessor=dummy,
)


X_train_nbcv = nb_cv.fit_transform(X_train_lemma)
X_val_nbcv = nb_cv.transform(X_val_lemma)
X_test_nbcv = nb_cv.transform(X_test_lemma)


In [15]:
bnb = naive_bayes.BernoulliNB()
bnb.fit(X_train_nbcv, y_train)
y_pred = bnb.predict(X_train_nbcv)
print((metrics.classification_report(y_train, y_pred)))


              precision    recall  f1-score   support

         0.0       0.65      0.62      0.64     45187
         1.0       0.58      0.61      0.59     38631

    accuracy                           0.62     83818
   macro avg       0.62      0.62      0.62     83818
weighted avg       0.62      0.62      0.62     83818



In [16]:
y_pred = bnb.predict(X_test_nbcv)
print((metrics.classification_report(y_test, y_pred)))

              precision    recall  f1-score   support

         0.0       0.63      0.61      0.62      5509
         1.0       0.57      0.60      0.59      4839

    accuracy                           0.61     10348
   macro avg       0.60      0.61      0.60     10348
weighted avg       0.61      0.61      0.61     10348



In [17]:
# feature_probs = bnb.feature_log_prob_[1,:]
# feature_names = nb_cv.get_feature_names_out()

feat_imp = pd.DataFrame(
    data={
        "term": nb_cv.get_feature_names_out(),
        "neg_log_proba": bnb.feature_log_prob_[0, :],
        "pos_log_proba": bnb.feature_log_prob_[1, :],
    }
)
feat_imp["log_diff"] = feat_imp["pos_log_proba"] - feat_imp["neg_log_proba"]
feat_imp.sort_values("log_diff", ascending=False)[:20]


,term,neg_log_proba,pos_log_proba,log_diff
9339,reid,-9.109171,-6.848290,2.260881
7937,not_limite,-8.926850,-6.950944,1.975905
8320,outreach,-8.772699,-6.848290,1.924409
2719,deadline_is,-8.639167,-6.872983,1.766185
2629,cutting_myself,-8.416024,-6.824192,1.591831
2389,contest_is,-8.521384,-6.950944,1.570440
11349,they_deserve,-8.153660,-6.630036,1.523623
7379,my_depression,-6.074218,-4.560447,1.513771
9952,severe_depression,-7.583115,-6.119211,1.463904
12624,wellbutrin,-7.386404,-5.997514,1.388891


In [18]:
feat_imp.sort_values("log_diff", ascending=True)[:20]


,term,neg_log_proba,pos_log_proba,log_diff
751,apply_or_read,-3.543119,-10.561862,-7.018743
4142,full_report,-4.167529,-10.561862,-6.394333
8818,power_score,-4.171824,-10.561862,-6.390039
5816,is_searching,-4.287278,-10.561862,-6.274584
5148,hiring_data,-4.362501,-10.561862,-6.199361
2671,data_analyst,-4.482239,-10.561862,-6.079623
4003,following_skills,-4.555294,-10.561862,-6.006568
11934,type_attractor,-4.794353,-10.561862,-5.767509
9303,reddit_stream,-4.802407,-10.561862,-5.759455
8883,previous_discussion,-4.892609,-10.561862,-5.669253


### Logistic Regression

In [20]:
def dummy(doc):
    return doc


lr_tfidf = feature_extraction.text.TfidfVectorizer(
    min_df=41,
    binary=True,
    tokenizer=dummy,
    preprocessor=dummy,
    norm='l2'
)


X_train_lrtfidf = lr_tfidf.fit_transform(X_train_lemma)
X_val_lrttfidf = lr_tfidf.transform(X_val_lemma)
X_test_lrtfidf = lr_tfidf.transform(X_test_lemma)


In [21]:
lr = linear_model.LogisticRegression(C=0.1)
lr.fit(X_train_lrtfidf,y_train)
y_pred = lr.predict(X_train_lrtfidf)
print((metrics.classification_report(y_train, y_pred)))

              precision    recall  f1-score   support

         0.0       0.64      0.78      0.70     45187
         1.0       0.65      0.49      0.56     38631

    accuracy                           0.65     83818
   macro avg       0.65      0.64      0.63     83818
weighted avg       0.65      0.65      0.64     83818



In [22]:
y_pred = lr.predict(X_test_lrtfidf)
print((metrics.classification_report(y_test, y_pred)))

              precision    recall  f1-score   support

         0.0       0.61      0.74      0.67      5509
         1.0       0.61      0.46      0.53      4839

    accuracy                           0.61     10348
   macro avg       0.61      0.60      0.60     10348
weighted avg       0.61      0.61      0.60     10348



In [23]:
feat_imp = pd.DataFrame(
    data={
        "term": lr_tfidf.get_feature_names_out(),
        "coef": lr.coef_[0, :],
    }
)
feat_imp.sort_values("coef", ascending=False)[:20]

,term,coef
2820,depression,2.566689
2818,depressed,1.232739
7476,my_life,1.173683
7379,my_depression,1.143291
2817,depress,1.038659
7446,my_head,0.952229
696,anymore,0.795105
3748,feel,0.776751
6665,low,0.763242
10890,suicide,0.758173


In [24]:
feat_imp.sort_values("coef", ascending=True)[:20]

,term,coef
751,apply_or_read,-2.155555
7178,more_here,-2.143103
4142,full_report,-1.891967
8818,power_score,-1.883302
4483,google_map,-1.786430
11565,thread,-1.330726
5816,is_searching,-1.154282
5799,is_looke,-1.130732
11934,type_attractor,-1.109782
11837,trip_report,-1.108816


### Random Forest

In [52]:
def dummy(doc):
    return doc


rf_cv = feature_extraction.text.CountVectorizer(
    min_df=min_docfreq,
    binary=False,
    tokenizer=dummy,
    preprocessor=dummy,
)


X_train_rfcv = rf_cv.fit_transform(X_train_lemma)
X_val_rfcv = rf_cv.transform(X_val_lemma)
X_test_rfcv = rf_cv.transform(X_test_lemma)


In [199]:
rf = ensemble.RandomForestClassifier(n_estimators=300, max_depth=int(np.log2(X_train.shape[0])*2), max_features="sqrt", n_jobs=-1)
rf.fit(X_train_rfcv, y_train)
y_pred = rf.predict(X_train_rfcv)
print((metrics.classification_report(y_train, y_pred)))


              precision    recall  f1-score   support

         0.0       0.63      0.96      0.76     45476
         1.0       0.89      0.35      0.50     38812

    accuracy                           0.68     84288
   macro avg       0.76      0.66      0.63     84288
weighted avg       0.75      0.68      0.64     84288



In [200]:
y_pred = rf.predict(X_test_rfcv)
print((metrics.classification_report(y_test, y_pred)))


              precision    recall  f1-score   support

         0.0       0.57      0.88      0.69      5521
         1.0       0.65      0.26      0.37      4886

    accuracy                           0.59     10407
   macro avg       0.61      0.57      0.53     10407
weighted avg       0.61      0.59      0.54     10407



In [203]:
feat_imp = pd.DataFrame(
    data={
        "term": rf_cv.get_feature_names_out(),
        "importance": rf.feature_importances_,
    }
)
feat_imp.sort_values("importance", ascending=False)[:20]

,term,importance
1063,apply_or_read,0.021331
3083,depression,0.017523
7405,more_here,0.014361
4418,full_report,0.011338
4748,google_map,0.010663
6040,is_looke,0.010147
7694,my_life,0.009345
9963,score,0.008892
6061,is_searching,0.007986
4277,following_skills,0.007701


### Gradient Boosted

In [209]:
gbc = ensemble.RandomForestClassifier(n_estimators=300, max_depth=32)
gbc.fit(X_train_rfcv, y_train)
y_pred = gbc.predict(X_train_rfcv)
print((metrics.classification_report(y_train, y_pred)))

              precision    recall  f1-score   support

         0.0       0.64      0.96      0.76     45476
         1.0       0.88      0.35      0.51     38812

    accuracy                           0.68     84288
   macro avg       0.76      0.66      0.64     84288
weighted avg       0.75      0.68      0.65     84288



In [210]:
y_pred = gbc.predict(X_test_rfcv)
print((metrics.classification_report(y_test, y_pred)))


              precision    recall  f1-score   support

         0.0       0.57      0.87      0.69      5521
         1.0       0.65      0.26      0.38      4886

    accuracy                           0.59     10407
   macro avg       0.61      0.57      0.53     10407
weighted avg       0.61      0.59      0.54     10407



In [211]:
feat_imp = pd.DataFrame(
    data={
        "term": rf_cv.get_feature_names_out(),
        "importance": gbc.feature_importances_,
    }
)
feat_imp.sort_values("importance", ascending=False)[:20]

,term,importance
7405,more_here,0.019773
1063,apply_or_read,0.018482
3083,depression,0.017385
4418,full_report,0.013166
4748,google_map,0.011787
4277,following_skills,0.009062
7694,my_life,0.008240
6040,is_looke,0.008215
9963,score,0.007397
6061,is_searching,0.007017


In [212]:
## TUNE WITH https://stackoverflow.com/questions/31948879/using-explicit-predefined-validation-set-for-grid-search-with-sklearn

## LDA

### Train LDA

In [18]:
id2word = corpora.Dictionary(X_train_lemma)
train_corpus = [id2word.doc2bow(text) for text in X_train_lemma]
val_corpus = [id2word.doc2bow(text) for text in X_val_lemma]
test_corpus = [id2word.doc2bow(text) for text in X_test_lemma]

In [19]:
lda_model = gensim.models.ldamulticore.LdaMulticore(
    corpus=train_corpus,
    id2word=id2word,
    num_topics=40, 
    random_state=100,
    workers=5,
    passes=3,
    alpha=1.0,
    per_word_topics=True,
    minimum_probability=0
)

In [20]:
print(lda_model.print_topics())

[(15, '0.019*"music" + 0.006*"song" + 0.004*"time" + 0.003*"think" + 0.003*""" + 0.003*"spotify_apple" + 0.003*"feel" + 0.002*"love" + 0.002*"game" + 0.002*"play"'), (3, '0.009*"score" + 0.009*"game" + 0.004*"play" + 0.004*"single" + 0.003*"time" + 0.003*"leave" + 0.003*"game_score" + 0.003*""" + 0.003*"field" + 0.003*"feel"'), (36, '0.009*"eat" + 0.004*"food" + 0.003*"feel" + 0.003*"time" + 0.002*"help" + 0.002*"good" + 0.002*""" + 0.002*"drink" + 0.002*"week" + 0.002*"start"'), (1, '0.005*"time" + 0.004*"feel" + 0.004*"think" + 0.003*"people" + 0.003*"feel_like" + 0.003*"good" + 0.002*"help" + 0.002*"look" + 0.002*"life" + 0.002*"ask"'), (6, '0.004*"big" + 0.003*"power" + 0.003*"good" + 0.003*"star" + 0.003*"gt" + 0.003*"key" + 0.003*"world" + 0.003*"actor" + 0.002*"bar" + 0.002*"hero"'), (4, '0.003*"people" + 0.003*"feel" + 0.003*"good" + 0.003*"love" + 0.003*"time" + 0.002*"guy" + 0.002*"think" + 0.002*"movie" + 0.002*"show" + 0.002*"come"'), (21, '0.005*"writer" + 0.005*"good" + 0

In [111]:
pprint(lda_model.print_topic(0))

('0.004*"play" + 0.003*"time" + 0.003*"join" + 0.003*"think" + 0.003*"good" + '
 '0.003*"player" + 0.002*"people" + 0.002*"live" + 0.002*"game" + 0.002*"feel"')


In [41]:
print("Train Perpelexity: ", lda_model.log_perplexity(train_corpus))
print("Val Perpelexity: ", lda_model.log_perplexity(val_corpus))


Train Perpelexity:  -11.361446822016653
Val Perpelexity:  -14.487513244613563


In [227]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=X_train_lemma, dictionary=id2word, coherence='c_v')
print("Train Coherence: ", coherence_model_lda.get_coherence())
coherence_model_lda = CoherenceModel(model=lda_model, texts=X_val_lemma, dictionary=id2word, coherence='c_v')
print("Val Coherence: ", coherence_model_lda.get_coherence())

# http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf

Train Coherence:  0.4447176987243576
Val Coherence:  0.3916391883952768


In [135]:
id2word.id2token[246]

'friend'

In [159]:
topic_terms = []
for i in range(40):
    topic_terms.append({
        "topic": i,
        "terms": ", ".join([id2word.id2token[x[0]] for x in lda_model.get_topic_terms(i, 20)])
    })

topic_terms = pd.DataFrame(topic_terms)

## Visualize LDA

In [21]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, train_corpus, id2word, sort_topics=False)
vis

C:\Users\yagne\anaconda3\envs\common\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.003422 -0.000031       1        1  2.335713
1     -0.014430 -0.001463       2        1  2.346752
2     -0.014519 -0.010414       3        1  2.384948
3      0.020811 -0.003195       4        1  2.324363
4     -0.007848 -0.001895       5        1  2.282307
5     -0.016918 -0.005535       6        1  2.304066
6      0.019814 -0.001126       7        1  2.572910
7     -0.023404 -0.012960       8        1  2.619397
8     -0.014777  0.001928       9        1  2.323432
9     -0.018620 -0.003914      10        1  2.483272
10     0.012453 -0.010768      11        1  2.601982
11     0.006194 -0.011820      12        1  2.139363
12    -0.013257 -0.007810      13        1  2.341737
13    -0.027229 -0.010723      14        1  2.830261
14    -0.014585 -0.002507      15        1  2.297966
15    -0.008381 -0.010527      16        1  2.326320
16    -0.004282 -0.000054      17        1  2.142727
17    -0.007937  0.002761      18        1  2.376048
18    -0.017087 -0.007445      19        1  2.381048
19     0.015415  0.004553      20        1  2.237482
20     0.006464  0.175744      21        1  2.774979
21    -0.009400 -0.003908      22        1  2.357874
22    -0.012952 -0.006176      23        1  2.256280
23     0.000929  0.015138      24        1  2.403336
24    -0.014740 -0.010716      25        1  2.381302
25    -0.020689 -0.010171      26        1  2.730558
26    -0.002838  0.005023      27        1  2.179884
27    -0.016262 -0.008288      28        1  2.472329
28    -0.025368 -0.012571      29        1  2.557926
29    -0.003712 -0.012492      30        1  2.801574
30     0.012763  0.023730      31        1  3.342577
31    -0.000494  0.027125      32        1  2.235660
32    -0.006621 -0.002403      33        1  2.024437
33    -0.013751 -0.002543      34        1  2.305149
34     0.003550 -0.001231      35        1  2.152821
35     0.278401 -0.016488      36        1  2.879008
36    -0.028785 -0.021841      37        1  3.104753
37    -0.003636  0.003127      38        1  2.213530
38    -0.043692 -0.024946      39        1  5.207739
39     0.025998 -0.023168      40        1  1.966190, topic_info=               Term          Freq         Total Category  logprob  loglift
0                 "  19491.000000  19491.000000  Default  30.0000  30.0000
3538          music   3247.000000   3247.000000  Default  29.0000  29.0000
4778         assist   1557.000000   1557.000000  Default  28.0000  28.0000
2168  apply_or_read   1304.000000   1304.000000  Default  27.0000  27.0000
2194      more_here   1309.000000   1309.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
8183       wish_you    322.385257    421.397879  Topic40  -5.7235   3.6612
4002       help_you    333.675613    479.419941  Topic40  -5.6891   3.5667
8175         stream    310.764667    603.459996  Topic40  -5.7602   3.2654
4820        keep_it    313.647061    735.263712  Topic40  -5.7510   3.0771
211           state    312.110473   2683.595937  Topic40  -5.7559   1.7775

[3444 rows x 6 columns], token_table=       Topic      Freq Term
term                       
0          1  0.008003    "
0          2  0.011800    "
0          3  0.018316    "
0          4  0.017187    "
0          5  0.011800    "
...      ...       ...  ...
16954     27  0.011913    🙏
16954     32  0.011913    🙏
16954     33  0.011913    🙏
16954     39  0.011913    🙏
16954     40  0.011913    🙏

[29879 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40])

In [161]:
topic_terms["marginal_topic_distribution"] = vis.to_dict()['mdsDat']['Freq']


### Transform

In [53]:
X_train_lda = lda_model.get_document_topics(train_corpus, minimum_probability=0.01, per_word_topics=False)
X_train_lda = gensim.matutils.corpus2csc(X_train_lda).transpose()

In [54]:
X_val_lda = lda_model.get_document_topics(val_corpus, minimum_probability=0.01, per_word_topics=False)
X_val_lda = gensim.matutils.corpus2csc(X_val_lda).transpose()

In [55]:
X_test_lda = lda_model.get_document_topics(test_corpus, minimum_probability=0.01, per_word_topics=False)
X_test_lda = gensim.matutils.corpus2csc(X_test_lda).transpose()

### Logistic Regression

In [77]:
from sklearn.preprocessing import normalize

X_train_lda_dense = X_train_lda.toarray()
X_train_lda_norm = (X_train_lda_dense - X_train_lda_dense.min(0)) / X_train_lda_dense.ptp(0) 
X_val_lda_norm = (X_val_lda.toarray() - X_train_lda_dense.min(0)) / X_train_lda_dense.ptp(0) 
X_test_lda_norm = (X_test_lda.toarray() - X_train_lda_dense.min(0)) / X_train_lda_dense.ptp(0) 

In [83]:
lr_lda = linear_model.LogisticRegression(C=0.1)
lr_lda.fit(X_train_lda_norm,y_train)
y_pred = lr_lda.predict(X_train_lda_norm)
print((metrics.classification_report(y_train, y_pred)))

              precision    recall  f1-score   support

         0.0       0.59      0.73      0.65     45476
         1.0       0.55      0.40      0.47     38812

    accuracy                           0.58     84288
   macro avg       0.57      0.56      0.56     84288
weighted avg       0.57      0.58      0.56     84288



In [84]:
y_pred = lr_lda.predict(X_test_lda_norm)
print((metrics.classification_report(y_test, y_pred)))

              precision    recall  f1-score   support

         0.0       0.58      0.73      0.65      5521
         1.0       0.57      0.40      0.47      4886

    accuracy                           0.58     10407
   macro avg       0.57      0.57      0.56     10407
weighted avg       0.57      0.58      0.56     10407



In [131]:
feat_imp = pd.DataFrame(
    data={
        "term": range(1,41),
        "coef": lr_lda.coef_[0, :],
    }
)
feat_imp.sort_values("term", ascending=True)

,term,coef
0,1,-0.057540
1,2,1.524415
2,3,0.930218
3,4,-2.217743
4,5,0.010867
5,6,1.156423
6,7,-1.464335
7,8,1.451992
8,9,1.801334
9,10,1.120321


In [157]:
lr_lda.coef_[0,:]

array([-0.05753972,  1.52441532,  0.93021825, -2.21774323,  0.01086727,
        1.15642313, -1.46433531,  1.45199242,  1.80133413,  1.12032054,
       -0.99192986,  0.98274882,  1.27421381,  0.3494273 ,  1.04753454,
        0.80800978,  0.04487261,  0.49181669,  1.59876632, -0.36867661,
       -3.40672334,  0.40916922,  1.11977618, -0.54266428,  0.74152127,
        1.61270525, -0.53792285,  1.43143431,  1.59798763, -0.5283352 ,
       -0.26673444,  0.73573048,  0.62648611,  0.78835484, -0.4207113 ,
       -2.83745918,  0.41957392,  0.72486662,  0.48524158, -2.17360655])

In [162]:
topic_terms["coef"] = lr_lda.coef_[0,:]
topic_terms.to_csv("topic_results.csv",index=False)

### Gradient Boosting

In [ ]:
gbc_lda = ensemble.GradientBoostingClassifier(n_estimators=100, max_depth=8)
gbc_lda.fit(X_train_lda, y_train)
y_pred = gbc_lda.predict(X_train_lda)
print((metrics.classification_report(y_train, y_pred)))

              precision    recall  f1-score   support

         0.0       0.63      0.71      0.67     45476
         1.0       0.60      0.52      0.56     38812

    accuracy                           0.62     84288
   macro avg       0.62      0.61      0.61     84288
weighted avg       0.62      0.62      0.62     84288



In [ ]:
y_pred = gbc_lda.predict(X_test_lda)
print((metrics.classification_report(y_test, y_pred)))

              precision    recall  f1-score   support

         0.0       0.60      0.68      0.64      5521
         1.0       0.58      0.49      0.53      4886

    accuracy                           0.59     10407
   macro avg       0.59      0.59      0.59     10407
weighted avg       0.59      0.59      0.59     10407



In [ ]:
feat_imp = pd.DataFrame(
    data={
        "term": range(1,21),
        "importance": gbc_lda.feature_importances_,
    }
)
feat_imp.sort_values("importance", ascending=False)[:20]

,term,importance
14,15,0.213426
8,9,0.128100
9,10,0.091999
18,19,0.076834
5,6,0.065119
3,4,0.042905
16,17,0.042767
11,12,0.042299
17,18,0.040733
12,13,0.037137


## Combined BoW + LDA

### Logistic Regression

In [70]:
X_train_lda_lrbow = hstack((X_train_lda, X_train_lrtfidf))
X_test_lda_lrbow =  hstack((X_test_lda, X_test_lrtfidf))

lr_lda_bow = linear_model.LogisticRegression(C=0.2)
lr_lda_bow.fit(X_train_lda_lrbow,y_train)
y_pred = lr_lda_bow.predict(X_train_lda_lrbow)
print((metrics.classification_report(y_train, y_pred)))

              precision    recall  f1-score   support

         0.0       0.67      0.75      0.71     45476
         1.0       0.66      0.56      0.61     38812

    accuracy                           0.67     84288
   macro avg       0.67      0.66      0.66     84288
weighted avg       0.67      0.67      0.66     84288



In [71]:
y_pred = lr_lda_bow.predict(X_test_lda_lrbow)
print((metrics.classification_report(y_test, y_pred)))

              precision    recall  f1-score   support

         0.0       0.62      0.70      0.66      5521
         1.0       0.60      0.51      0.55      4886

    accuracy                           0.61     10407
   macro avg       0.61      0.60      0.60     10407
weighted avg       0.61      0.61      0.61     10407



### Random Forest

In [65]:
X_train_lda_rfbow = hstack((X_train_lda, X_train_rfcv))
X_test_lda_rfbow =  hstack((X_test_lda, X_test_rfcv))

rf_lda_bow = ensemble.RandomForestClassifier(n_estimators=500, max_depth=int(np.log2(X_train.shape[0])*4), max_features="sqrt", n_jobs=-1)
rf_lda_bow.fit(X_train_lda_rfbow, y_train)
y_pred = rf_lda_bow.predict(X_train_lda_rfbow)
print((metrics.classification_report(y_train, y_pred)))

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     45476
         1.0       1.00      0.97      0.98     38812

    accuracy                           0.99     84288
   macro avg       0.99      0.98      0.99     84288
weighted avg       0.99      0.99      0.99     84288



In [66]:
y_pred = rf_lda_bow.predict(X_test_lda_rfbow)
print((metrics.classification_report(y_test, y_pred)))

              precision    recall  f1-score   support

         0.0       0.63      0.67      0.65      5521
         1.0       0.60      0.55      0.57      4886

    accuracy                           0.61     10407
   macro avg       0.61      0.61      0.61     10407
weighted avg       0.61      0.61      0.61     10407



### Gradient Boosting

In [68]:
X_train_lda_rfbow = hstack((X_train_lda, X_train_rfcv))
X_test_lda_rfbow =  hstack((X_test_lda, X_test_rfcv))

gbc_lda_bow = ensemble.GradientBoostingClassifier(n_estimators=200, max_depth=5)
gbc_lda_bow.fit(X_train_lda_lrbow,y_train)
y_pred = gbc_lda_bow.predict(X_train_lda_lrbow)
print((metrics.classification_report(y_train, y_pred)))

              precision    recall  f1-score   support

         0.0       0.67      0.75      0.71     45476
         1.0       0.66      0.57      0.61     38812

    accuracy                           0.67     84288
   macro avg       0.67      0.66      0.66     84288
weighted avg       0.67      0.67      0.67     84288



In [69]:
y_pred = gbc_lda_bow.predict(X_test_lda_rfbow)
print((metrics.classification_report(y_test, y_pred)))

              precision    recall  f1-score   support

         0.0       0.61      0.65      0.63      5521
         1.0       0.58      0.54      0.55      4886

    accuracy                           0.60     10407
   macro avg       0.59      0.59      0.59     10407
weighted avg       0.60      0.60      0.60     10407

